In [ ]:
import ipywidgets as W
from IPython.display import display, clear_output
from tradle_guesser import load_country_data, best_guesses

# Load data
print("Loading country data...")
centroid_list, distance_df = load_country_data()
countries = sorted(centroid_list["Country"].unique().tolist())
dir_choices = ["", "N", "S", "E", "W", "NE", "NW", "SE", "SW"]
print(f"Loaded {len(countries)} countries")

In [ ]:
rows_box = W.VBox([])  # will hold hint rows

def make_row():
    return W.HBox([
        W.Dropdown(options=countries, description="Country", layout=W.Layout(width="300px")),
        W.BoundedIntText(value=1000, min=0, max=40075, step=1, description="Distance (km)", layout=W.Layout(width="150px")),
        W.Dropdown(options=dir_choices, value="", description="Direction", layout=W.Layout(width="150px")),
        W.Button(description="−", button_style="", layout=W.Layout(width="40px"))
    ])

def add_row(_=None):
    row = make_row()
    def remove_row(btn):
        rows = list(rows_box.children)
        rows.remove(row)
        rows_box.children = tuple(rows)
    row.children[-1].on_click(remove_row)
    rows_box.children = (*rows_box.children, row)

add_btn = W.Button(description="Add hint", button_style="info", layout=W.Layout(width="100px"))
add_btn.on_click(add_row)
add_row()  # start with one

penalty = W.BoundedFloatText(value=10.0, min=0, max=1e6, step=1.0, description="Penalty", layout=W.Layout(width="120px"))
tolerance = W.BoundedFloatText(value=0.0, min=0, max=90, step=0.5, description="Tolerance (°)", layout=W.Layout(width="130px"))
topn    = W.BoundedIntText(value=15, min=1, max=250, step=1, description="Top N", layout=W.Layout(width="110px"))

run_btn = W.Button(description="Compute", button_style="success", layout=W.Layout(width="100px"))
out = W.Output()

def run(_):
    hints = []
    for row in rows_box.children:
        ctry = row.children[0].value
        km   = row.children[1].value
        d    = row.children[2].value or None
        hints.append((ctry, km) if d is None else (ctry, km, d))
    with out:
        clear_output(wait=True)
        if not hints:
            print("Add at least one hint.")
            return
        try:
            res = best_guesses(hints, centroid_list, distance_df, 
                             penalty=float(penalty.value), 
                             tol=float(tolerance.value))
            display(res.head(int(topn.value)))
        except Exception as e:
            print(f"Error: {e}")

run_btn.on_click(run)

implementation_details = W.HTML("""
<div style="margin-top: 30px; padding: 15px; background-color: #f5f5f5; border-left: 4px solid #2196F3;">
    <h3 style="margin-top: 0;">Implementation Details</h3>
    <p><strong>How the ranking works:</strong></p>
    <ol>
        <li><strong>Distance Error:</strong> For each candidate country, we calculate the absolute difference 
            between its actual distance from your guess and the distance Tradle reported.</li>
        <li><strong>Direction Filtering:</strong> We use simple latitude/longitude comparisons to check if a country 
            is in the correct direction. Countries in the wrong direction have their error multiplied by the penalty.
            <br><em>Note: This uses simple lat/lon comparison, not great circle bearings, which may be inaccurate 
            for long distances or near the poles.</em></li>
        <li><strong>Tolerance:</strong> Adds a buffer zone (in degrees) around direction boundaries. 
            For example, tolerance=5° means "West" includes anything 5° or more west.</li>
        <li><strong>Total Error:</strong> Sum the (possibly penalized) distance errors across all your hints. 
            Lower total error = better match.</li>
    </ol>
    <p><strong>Distance Calculation:</strong> Uses the Haversine formula to compute great circle distances 
    between country centroids (from Natural Earth data, projected to EPSG:6933 for accuracy).</p>
    <p><strong>Output:</strong> The <code>adjusted_km_total_error</code> column shows the total error in kilometers. 
    Countries are sorted with lowest error first.</p>
</div>
""")

ui = W.VBox([
    W.HTML("<h2>Tradle Geographic Triangulator</h2>"),
    W.HTML("<p><strong>How to use:</strong> For each guess you made in Tradle, enter the country name, "
           "the distance shown, and the direction. <em>Note: The direction is FROM your guess TO the actual country.</em></p>"),
    W.HBox([add_btn, penalty, tolerance, topn, run_btn]),
    rows_box,
    out,
    implementation_details
])

display(ui)